# Predicciones Finales - Competencia CAS y ACTEX 2025

## Metodología
Para cada estudiante en el conjunto de validación:
1. **Por cada cobertura** (Contenidos, Adicionales, Médicos, RC):
   - **Clasificación**: ¿Tuvo siniestro? (0 o 1)
   - **Frecuencia**: ¿Cuántos siniestros? (si clasificación = 1)
     - Contenidos: modelo predictivo
     - Adicionales: **constante 1.0** (97.4% histórico)
     - Médicos: modelo predictivo (corregido)
     - RC: **constante 1.0** (100% histórico)
   - **Severidad**: ¿Cuánto costó? (si clasificación = 1)
   - **Monto cobertura = clasificación × frecuencia × severidad**
2. **Predicción total = Suma de las 4 coberturas**

## Modelos utilizados
- **10 modelos predictivos** (clasificación + severidad para todas, frecuencia para Contenidos y Médicos)
- **2 frecuencias constantes** (Adicionales = 1.0, RC = 1.0)

In [109]:
# Imports
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, ClassifierMixin
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas")

✓ Librerías importadas


## 1. Definir Clase ConstantFrequencyModel

In [110]:
class ConstantFrequencyModel(BaseEstimator, ClassifierMixin):
    """Modelo que siempre predice frecuencia = 1.0 (usado para RC)"""
    
    def __init__(self, constant_value=1.0):
        self.constant_value = constant_value
    
    def fit(self, X, y):
        unique_values = np.unique(y)
        if len(unique_values) == 1:
            self.constant_value = unique_values[0]
        else:
            self.constant_value = np.mean(y)
        self.classes_ = unique_values
        return self
    
    def predict(self, X):
        return np.full(X.shape[0] if hasattr(X, 'shape') else len(X), self.constant_value)

print("✓ Clase ConstantFrequencyModel definida")

✓ Clase ConstantFrequencyModel definida


## 2. Función para Extraer Modelo

Maneja las diferentes estructuras de modelos guardados.

In [111]:
def get_model_object(model_dict):
    """Extrae el objeto del modelo de la estructura guardada"""
    if isinstance(model_dict, dict):
        # Intentar diferentes claves
        if 'modelo' in model_dict:
            return model_dict['modelo']
        elif 'model' in model_dict:
            return model_dict['model']
        else:
            # Si no tiene ninguna clave conocida, asumir que el dict ES el modelo
            return model_dict
    else:
        # Si no es dict, asumir que es el modelo directamente
        return model_dict

def get_threshold(model_dict):
    """Extrae el threshold óptimo del modelo"""
    if isinstance(model_dict, dict):
        if 'threshold_optimo' in model_dict:
            return model_dict['threshold_optimo']
        elif 'threshold' in model_dict:
            return model_dict['threshold']
    return 0.5  # Default si no encuentra threshold

def predict_with_threshold(modelo_raw, X_prep):
    """
    Aplica predicción usando threshold óptimo del modelo.
    Para modelos de clasificación solamente.
    """
    modelo = get_model_object(modelo_raw)
    threshold = get_threshold(modelo_raw)
    
    # Obtener probabilidades
    if hasattr(modelo, 'predict_proba'):
        y_proba = modelo.predict_proba(X_prep)[:, 1]
    elif hasattr(modelo, 'decision_function'):
        y_score = modelo.decision_function(X_prep)
        # Normalizar scores a [0,1]
        y_proba = (y_score - y_score.min()) / (y_score.max() - y_score.min() + 1e-10)
    else:
        # Si no tiene predict_proba, usar predict directo
        return modelo.predict(X_prep)
    
    # Aplicar threshold
    return (y_proba >= threshold).astype(int)

print("✓ Funciones de extracción y threshold definidas")

✓ Funciones de extracción y threshold definidas


## 3. Carga de Datos

In [112]:
# Cargar dataset completo
df_full = pd.read_csv('../data/input/input.csv')

print(f"Dataset completo: {df_full.shape}")
print(f"\nDistribución de retención:")
print(df_full['retencion'].value_counts())

# Filtrar solo validación
df_validacion = df_full[df_full['retencion'] == 'validacion'].copy()
print(f"\nDataset validación: {df_validacion.shape}")

# Guardar IDs
ids_validacion = df_validacion['estudiante_id'].values
print(f"IDs de validación: {len(ids_validacion)}")

df_validacion.head()

Dataset completo: (10000, 20)

Distribución de retención:
retencion
entrenamiento    7999
validacion       2001
Name: count, dtype: int64

Dataset validación: (2001, 20)
IDs de validación: 2001


,estudiante_id,nombre,año_cursado,estudios_area,calif_promedio,2_o_mas_inquilinos,en_campus,distancia_al_campus,genero,extintor_incendios,clase_suscripcion,retencion,Gastos_Adicionales_siniestros_num,Gastos_Adicionales_siniestros_monto,Gastos_Medicos_RC_siniestros_num,Gastos_Medicos_RC_siniestros_monto,Resp_Civil_siniestros_num,Resp_Civil_siniestros_monto,Contenidos_siniestros_num,Contenidos_siniestros_monto
7,8,"el-Sala, Khaleel",4to año,Humanidades,7.01,No,Fuera de campus,1.111582,Masculino,Si,Clase3,validacion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,16,"Fenton, Danielle",2do año,Administracion,1.87,No,Dentro de campus,0.000000,Masculino,Si,Clase1,validacion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,31,"Williams, Shaquelle",4to año,Ciencias,4.15,No,Fuera de campus,0.462273,Femenino,No,Clase3,validacion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,49,"Diaz, Rance",1er año,Otro,3.56,No,Dentro de campus,0.000000,Femenino,Si,Clase1,validacion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,56,"Law, Shawde",1er año,Otro,5.56,No,Dentro de campus,0.000000,Masculino,Si,Clase1,validacion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Carga de Modelos

In [113]:
models_path = '../models/'

print("Cargando 11 modelos (frecuencia adicionales y RC son constantes)...\n")

# Cargar modelos raw (estructuras completas)
modelos_raw = {}
archivos_modelos = [
    ('contenidos_clasificacion', 'clasificacion_contenidos.pkl'),
    ('contenidos_frecuencia', 'frecuencia_contenidos.pkl'),
    ('contenidos_severidad', 'severidad_contenidos.pkl'),
    ('adicionales_clasificacion', 'clasificacion_adicionales.pkl'),
    # adicionales_frecuencia: NO se carga - usamos constante 1.0 (97.4% de casos)
    ('adicionales_severidad', 'severidad_adicionales.pkl'),
    ('medicos_clasificacion', 'clasificacion_medicos_best_model.pkl'),
    ('medicos_frecuencia', 'frecuencia_medicos_best_model.pkl'),  # CORREGIDO
    ('medicos_severidad', 'severidad_medicos_extratreesregressor_model.pkl'),
    ('rc_clasificacion', 'clasificacion_rc_best_model.pkl'),
    ('rc_frecuencia', 'frecuencia_rc_constant_model.pkl'),
    ('rc_severidad', 'severidad_rc_model.pkl'),
]

for nombre, archivo in archivos_modelos:
    try:
        modelos_raw[nombre] = joblib.load(f'{models_path}{archivo}')
        print(f"✓ {nombre}")
    except Exception as e:
        print(f"✗ {nombre}: {str(e)[:60]}")

print(f"\n{len(modelos_raw)}/11 modelos cargados")
print("Nota: frecuencia_adicionales usa constante 1.0 (no requiere modelo)")

Cargando 11 modelos (frecuencia adicionales y RC son constantes)...

✓ contenidos_clasificacion
✓ contenidos_frecuencia
✓ contenidos_severidad
✓ adicionales_clasificacion
✓ adicionales_severidad
✓ medicos_clasificacion
✓ medicos_frecuencia
✓ medicos_severidad
✓ rc_clasificacion
✓ rc_frecuencia
✓ rc_severidad

11/11 modelos cargados
Nota: frecuencia_adicionales usa constante 1.0 (no requiere modelo)


## 5. Función Universal de Preprocesamiento

In [114]:
def preparar_datos_estandar(df, variables_seleccionadas):
    """
    Preprocesamiento estándar: OneHotEncoding + StandardScaler
    """
    categorical_features = ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']
    numeric_features = ['calif_promedio', 'distancia_al_campus']
    
    df_temp = df.copy()
    
    # OneHotEncoding (drop='first')
    df_encoded = pd.get_dummies(df_temp, columns=categorical_features, drop_first=True)
    
    # Estandarizar numéricas
    scaler = StandardScaler()
    for num_col in numeric_features:
        if num_col in df_encoded.columns:
            df_encoded[num_col] = scaler.fit_transform(df_encoded[[num_col]])
    
    # Agregar columnas faltantes con 0s
    for var in variables_seleccionadas:
        if var not in df_encoded.columns:
            df_encoded[var] = 0
    
    return df_encoded[variables_seleccionadas]

def preparar_datos_rc_severidad(df):
    """
    Preprocesamiento especial para RC severidad (usa LabelEncoder)
    """
    variables = ['2_o_mas_inquilinos', 'distancia_al_campus', 'genero', 
                 'extintor_incendios', 'Gastos_Medicos_RC_siniestros_num']
    
    df_temp = df.copy()
    # Agregar columna que no existe en validación
    df_temp['Gastos_Medicos_RC_siniestros_num'] = 0
    
    X = df_temp[variables].copy()
    
    # LabelEncoder para categóricas
    for col in X.columns:
        if X[col].dtype == 'object':
            le = LabelEncoder()
            X[col] = X[col].fillna('missing')
            X[col] = le.fit_transform(X[col])
    
    return X

print("✓ Funciones de preprocesamiento definidas")

✓ Funciones de preprocesamiento definidas


## 6. Variables Seleccionadas por Modelo

In [115]:
# Definir variables que cada modelo espera (después de preprocessing)
vars_modelos = {
    'contenidos_clasificacion': ['2_o_mas_inquilinos_Si', 'año_cursado_2do año', 'año_cursado_3er año', 
                                  'año_cursado_4to año', 'año_cursado_posgrado', 'distancia_al_campus'],
    
    'contenidos_frecuencia': ['2_o_mas_inquilinos_Si', 'año_cursado_2do año', 'año_cursado_3er año', 
                              'año_cursado_4to año', 'año_cursado_posgrado', 'calif_promedio', 
                              'distancia_al_campus', 'estudios_area_Ciencias', 'estudios_area_Humanidades', 
                              'estudios_area_Otro', 'extintor_incendios_Si', 'genero_Masculino', 
                              'genero_No respuesta', 'genero_Otro'],
    
    'contenidos_severidad': ['2_o_mas_inquilinos_Si', 'año_cursado_2do año', 'año_cursado_3er año', 
                             'año_cursado_4to año', 'año_cursado_posgrado', 'distancia_al_campus', 
                             'extintor_incendios_Si'],
    
    'adicionales_clasificacion': ['2_o_mas_inquilinos_Si', 'estudios_area_Ciencias', 'estudios_area_Humanidades', 
                                  'estudios_area_Otro', 'extintor_incendios_Si'],
    
    'adicionales_severidad': ['2_o_mas_inquilinos_Si', 'año_cursado_2do año', 'año_cursado_3er año', 
                              'año_cursado_4to año', 'año_cursado_posgrado', 'distancia_al_campus', 
                              'extintor_incendios_Si', 'genero_Masculino', 'genero_No respuesta', 'genero_Otro'],
    
    'medicos_clasificacion': ['distancia_al_campus', '2_o_mas_inquilinos_Si'],
    
    'medicos_frecuencia': ['calif_promedio', 'distancia_al_campus', 'año_cursado_2do año', 'año_cursado_3er año',
                           'año_cursado_4to año', 'año_cursado_posgrado', 'estudios_area_Ciencias', 
                           'estudios_area_Humanidades', 'estudios_area_Otro', 'genero_Masculino', 
                           'genero_No respuesta', 'genero_Otro', '2_o_mas_inquilinos_Si', 'extintor_incendios_Si'],
    
    'medicos_severidad': ['distancia_al_campus', 'año_cursado_1er año', 'año_cursado_posgrado', 
                          'estudios_area_Administracion', '2_o_mas_inquilinos_No', '2_o_mas_inquilinos_Si', 
                          'genero_Otro', 'extintor_incendios_No', 'extintor_incendios_Si'],
    
    'rc_clasificacion': ['distancia_al_campus', '2_o_mas_inquilinos_Si'],
}

print("✓ Variables definidas para 10 modelos (RC frecuencia es constante, RC severidad usa preprocesamiento especial)")

✓ Variables definidas para 10 modelos (RC frecuencia es constante, RC severidad usa preprocesamiento especial)


## 7. PREDICCIONES POR COBERTURA

### 7.1 CONTENIDOS

In [116]:
print("="*80)
print("COBERTURA: CONTENIDOS")
print("="*80)

# Clasificación CON THRESHOLD ÓPTIMO
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['contenidos_clasificacion'])
pred_cont_clas = predict_with_threshold(modelos_raw['contenidos_clasificacion'], X_prep)
threshold = get_threshold(modelos_raw['contenidos_clasificacion'])
print(f"Clasificación: {pred_cont_clas.sum()}/{len(pred_cont_clas)} siniestrados (threshold={threshold:.3f})")

# Frecuencia
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['contenidos_frecuencia'])
modelo = get_model_object(modelos_raw['contenidos_frecuencia'])
pred_cont_frec_raw = modelo.predict(X_prep)
pred_cont_frec = np.where(pred_cont_clas == 1, pred_cont_frec_raw, 0)
print(f"Frecuencia media (siniestrados): {pred_cont_frec[pred_cont_clas==1].mean():.3f}")

# Severidad
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['contenidos_severidad'])
modelo = get_model_object(modelos_raw['contenidos_severidad'])
pred_cont_sev_raw = np.maximum(modelo.predict(X_prep), 0)
pred_cont_sev = np.where(pred_cont_clas == 1, pred_cont_sev_raw, 0)
print(f"Severidad media (siniestrados): ${pred_cont_sev[pred_cont_clas==1].mean():,.2f}")

# Predicción final
pred_contenidos = pred_cont_clas * pred_cont_frec * pred_cont_sev
print(f"\n✓ Monto medio: ${pred_contenidos.mean():,.2f}")
print(f"  Monto total: ${pred_contenidos.sum():,.2f}")

COBERTURA: CONTENIDOS
Clasificación: 0/2001 siniestrados (threshold=0.560)
Frecuencia media (siniestrados): nan
Severidad media (siniestrados): $nan

✓ Monto medio: $0.00
  Monto total: $0.00


### 7.2 GASTOS ADICIONALES

In [117]:
print("="*80)
print("COBERTURA: GASTOS ADICIONALES")
print("="*80)

# Clasificación CON THRESHOLD ÓPTIMO
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['adicionales_clasificacion'])
pred_adic_clas = predict_with_threshold(modelos_raw['adicionales_clasificacion'], X_prep)
threshold = get_threshold(modelos_raw['adicionales_clasificacion'])
print(f"Clasificación: {pred_adic_clas.sum()}/{len(pred_adic_clas)} siniestrados (threshold={threshold:.3f})")

# Frecuencia (CONSTANTE = 1.0, ya que 97.4% de casos tienen frecuencia 1)
pred_adic_frec = np.where(pred_adic_clas == 1, 1.0, 0)
print(f"Frecuencia: 1.0 (constante - 97.4% de casos históricos)")

# Severidad
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['adicionales_severidad'])
modelo = get_model_object(modelos_raw['adicionales_severidad'])
pred_adic_sev_raw = np.maximum(modelo.predict(X_prep), 0)
pred_adic_sev = np.where(pred_adic_clas == 1, pred_adic_sev_raw, 0)
print(f"Severidad media (siniestrados): ${pred_adic_sev[pred_adic_clas==1].mean():,.2f}")

# Predicción final
pred_adicionales = pred_adic_clas * pred_adic_frec * pred_adic_sev
print(f"\n✓ Monto medio: ${pred_adicionales.mean():,.2f}")
print(f"  Monto total: ${pred_adicionales.sum():,.2f}")

COBERTURA: GASTOS ADICIONALES
Clasificación: 101/2001 siniestrados (threshold=0.100)
Frecuencia: 1.0 (constante - 97.4% de casos históricos)
Severidad media (siniestrados): $5,597.39

✓ Monto medio: $282.53
  Monto total: $565,336.24


### 7.3 GASTOS MÉDICOS RC

In [118]:
print("="*80)
print("COBERTURA: GASTOS MÉDICOS RC")
print("="*80)

# Clasificación CON THRESHOLD ÓPTIMO
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['medicos_clasificacion'])
pred_med_clas = predict_with_threshold(modelos_raw['medicos_clasificacion'], X_prep)
threshold = get_threshold(modelos_raw['medicos_clasificacion'])
print(f"Clasificación: {pred_med_clas.sum()}/{len(pred_med_clas)} siniestrados (threshold={threshold:.3f})")

# Frecuencia
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['medicos_frecuencia'])
modelo = get_model_object(modelos_raw['medicos_frecuencia'])
pred_med_frec_raw = modelo.predict(X_prep)
pred_med_frec = np.where(pred_med_clas == 1, pred_med_frec_raw, 0)
print(f"Frecuencia media (siniestrados): {pred_med_frec[pred_med_clas==1].mean():.3f}")

# Severidad
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['medicos_severidad'])
modelo = get_model_object(modelos_raw['medicos_severidad'])
pred_med_sev_raw = np.maximum(modelo.predict(X_prep), 0)
pred_med_sev = np.where(pred_med_clas == 1, pred_med_sev_raw, 0)
print(f"Severidad media (siniestrados): ${pred_med_sev[pred_med_clas==1].mean():,.2f}")

# Predicción final
pred_medicos = pred_med_clas * pred_med_frec * pred_med_sev
print(f"\n✓ Monto medio: ${pred_medicos.mean():,.2f}")
print(f"  Monto total: ${pred_medicos.sum():,.2f}")

COBERTURA: GASTOS MÉDICOS RC
Clasificación: 27/2001 siniestrados (threshold=0.650)
Frecuencia media (siniestrados): 1.148
Severidad media (siniestrados): $26,994.26

✓ Monto medio: $409.99
  Monto total: $820,381.89


### 7.4 RESPONSABILIDAD CIVIL

In [119]:
print("="*80)
print("COBERTURA: RESPONSABILIDAD CIVIL")
print("="*80)

# Clasificación CON THRESHOLD ÓPTIMO
X_prep = preparar_datos_estandar(df_validacion, vars_modelos['rc_clasificacion'])
pred_rc_clas = predict_with_threshold(modelos_raw['rc_clasificacion'], X_prep)
threshold = get_threshold(modelos_raw['rc_clasificacion'])
print(f"Clasificación: {pred_rc_clas.sum()}/{len(pred_rc_clas)} siniestrados (threshold={threshold:.3f})")

# Frecuencia (modelo constante = 1.0)
pred_rc_frec = np.where(pred_rc_clas == 1, 1.0, 0)
print(f"Frecuencia: 1.0 (constante)")

# Severidad (preprocesamiento especial con LabelEncoder)
X_prep = preparar_datos_rc_severidad(df_validacion)
modelo = get_model_object(modelos_raw['rc_severidad'])
pred_rc_sev_raw = np.maximum(modelo.predict(X_prep), 0)
pred_rc_sev = np.where(pred_rc_clas == 1, pred_rc_sev_raw, 0)
print(f"Severidad media (siniestrados): ${pred_rc_sev[pred_rc_clas==1].mean():,.2f}")

# Predicción final
pred_rc = pred_rc_clas * pred_rc_frec * pred_rc_sev
print(f"\n✓ Monto medio: ${pred_rc.mean():,.2f}")
print(f"  Monto total: ${pred_rc.sum():,.2f}")

COBERTURA: RESPONSABILIDAD CIVIL
Clasificación: 166/2001 siniestrados (threshold=0.650)
Frecuencia: 1.0 (constante)
Severidad media (siniestrados): $9,997.72

✓ Monto medio: $829.40
  Monto total: $1,659,622.26


## 8. Predicción Total y Estadísticas

In [120]:
# Sumar todas las coberturas
prediccion_total = pred_contenidos + pred_adicionales + pred_medicos + pred_rc

print("="*80)
print("PREDICCIÓN TOTAL")
print("="*80)
print(f"\nRegistros: {len(prediccion_total)}")
print(f"Media: ${prediccion_total.mean():,.2f}")
print(f"Mediana: ${np.median(prediccion_total):,.2f}")
print(f"Desv. Std: ${prediccion_total.std():,.2f}")
print(f"Mínimo: ${prediccion_total.min():,.2f}")
print(f"Máximo: ${prediccion_total.max():,.2f}")
print(f"Total: ${prediccion_total.sum():,.2f}")

print(f"\nContribución por cobertura:")
total = prediccion_total.sum()
print(f"  Contenidos: ${pred_contenidos.sum():,.2f} ({100*pred_contenidos.sum()/total:.1f}%)")
print(f"  Adicionales: ${pred_adicionales.sum():,.2f} ({100*pred_adicionales.sum()/total:.1f}%)")
print(f"  Médicos RC: ${pred_medicos.sum():,.2f} ({100*pred_medicos.sum()/total:.1f}%)")
print(f"  RC: ${pred_rc.sum():,.2f} ({100*pred_rc.sum()/total:.1f}%)")

PREDICCIÓN TOTAL

Registros: 2001
Media: $1,521.91
Mediana: $0.00
Desv. Std: $5,637.85
Mínimo: $0.00
Máximo: $61,911.08
Total: $3,045,340.39

Contribución por cobertura:
  Contenidos: $0.00 (0.0%)
  Adicionales: $565,336.24 (18.6%)
  Médicos RC: $820,381.89 (26.9%)
  RC: $1,659,622.26 (54.5%)


## 9. Preparar y Exportar Resultados

In [121]:
# Crear DataFrame final
df_predicciones = pd.DataFrame({
    'id': ids_validacion,
    'prediccion': prediccion_total
})

print(f"DataFrame creado: {df_predicciones.shape}")
print(f"\nPrimeras 10 filas:")
print(df_predicciones.head(10))

print(f"\nÚltimas 10 filas:")
print(df_predicciones.tail(10))

DataFrame creado: (2001, 2)

Primeras 10 filas:
    id   prediccion
0    8     0.000000
1   16  3222.258281
2   31     0.000000
3   49     0.000000
4   56  3131.241680
5   67     0.000000
6   83     0.000000
7   98     0.000000
8  102     0.000000
9  115     0.000000

Últimas 10 filas:
        id  prediccion
1991  9970         0.0
1992  9973         0.0
1993  9975         0.0
1994  9976         0.0
1995  9979         0.0
1996  9981         0.0
1997  9991         0.0
1998  9992         0.0
1999  9993         0.0
2000  9998         0.0


In [122]:
# Exportar a CSV
output_path = '../data/output/predicciones.csv'
df_predicciones.to_csv(output_path, index=False)

print("="*80)
print("✓ EXPORTACIÓN COMPLETADA")
print("="*80)
print(f"\nArchivo: {output_path}")
print(f"Registros: {len(df_predicciones)}")

# Verificar
import os
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path)
    print(f"Tamaño: {file_size:,} bytes")
    
    # Leer de vuelta
    df_verif = pd.read_csv(output_path)
    print(f"\nVerificación: {df_verif.shape}")
    print(f"Columnas: {list(df_verif.columns)}")
    print(f"\nPrimeras 5 filas:")
    print(df_verif.head())
    print("\n✓ Archivo verificado correctamente")

✓ EXPORTACIÓN COMPLETADA

Archivo: ../data/output/predicciones.csv
Registros: 2001
Tamaño: 21,492 bytes

Verificación: (2001, 2)
Columnas: ['id', 'prediccion']

Primeras 5 filas:
   id   prediccion
0   8     0.000000
1  16  3222.258281
2  31     0.000000
3  49     0.000000
4  56  3131.241680

✓ Archivo verificado correctamente
